In [1]:
# general imports
import sys
import os
path_to_vape = os.path.expanduser('~/Documents/code/Vape')
sys.path.append(path_to_vape)
sys.path.append(os.path.join(path_to_vape, 'jupyter'))
sys.path.append(os.path.join(path_to_vape, 'utils'))
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import utils_funcs as utils
import run_functions as rf
from subsets_analysis import Subsets
import pickle
import sklearn.decomposition
from cycler import cycler
import seaborn as sns
plt.rcParams['axes.prop_cycle'] = cycler(color=sns.color_palette('colorblind'))

#%run setup_notebook.ipynb

In [2]:
def get_trial_frames_single(clock, start, pre_frames, post_frames, fs=30, paq_rate=20000):

    # the frames immediately preceeding stim
    frame_idx = utils.closest_frame_before(clock, start)
    trial_frames = np.arange(frame_idx-pre_frames, frame_idx+post_frames)
    
    # is the trial outside of the frame clock
    is_beyond_clock = np.max(trial_frames) >= len(clock) or np.min(trial_frames) < 0
    
    if is_beyond_clock:
        return None
    
    frame_to_start = (start - clock[frame_idx]) / paq_rate  # time (s) from frame to trial_start
    frame_time_diff = np.diff(clock[trial_frames]) / paq_rate  # ifi (s)
    
    # did the function find the correct frame
    is_not_correct_frame = clock[frame_idx+1]  < start or clock[frame_idx] > start
    # the nearest frame to trial start was not during trial
    trial_not_running = frame_to_start > 1/fs
    # 
    frames_not_consecutive = np.max(frame_time_diff) > 1/(fs-1)
    
    if trial_not_running or frames_not_consecutive:
        return None
    return trial_frames

In [3]:
def build_flu_array_single(run, pre_frames=30, post_frames=80, fs=30):
    
    ''' Build an trial by trial fluoresence array of shape [n_cells x n_frames x n_trials]
        pre_frames = number of frames before stim to include in trial
        post_frames = number of frames after stim to include in trial
        fs = frame rate of imaging
        '''
    
    flu = run.flu
    # the frames that were actually imaged and the time (samples) that they occured
    clock = run.paqio_frames
    # the times of trial start in paq samples
    trial_start = run.spiral_start
    
    # check that the number of trial starts detected by x galvo thresholding
    # matches the number of trials reported by pycontrol
    assert len(trial_start) == len(run.trial_start)
    for i, start in enumerate(trial_start):
        trial_frames = get_trial_frames_single(clock, start, pre_frames, post_frames)       

        if trial_frames is None:
            flu_trial = np.full([flu.shape[0], pre_frames+post_frames], np.nan)
        else:
            flu_trial = flu[:, trial_frames]
            
        if i == 0:
            flu_array = flu_trial
        else:
            
            flu_array = np.dstack((flu_array, flu_trial))


    return np.swapaxes(flu_array,2,1)

In [4]:
class Session:
    def __init__(self, mouse, run_number, pkl_path, remove_nan_trials=True):
        self.mouse = mouse
        self.run_number = run_number
        self.pkl_path = pkl_path
        self.name = f'Mouse {mouse}, run {run_number}'
        self.run = None
        self.frequency = None
        
        self.load_data()
        self.define_s1_s2()
        if mouse=='J048' or mouse=='RL048':
            self.build_trials_multi()
        else:
            self.build_trials_single()
        self.label_trials()
        self.remove_nan_trials_inplace()
    
    def __str__(self):  
        """Define name"""
        return self.name
    
    def __repr__(self):
        """Define representation"""
        return f'instance {self.name} of Session class'
    
    def load_data(self, verbose=True):
        if verbose:
            print(f'Now loading mouse {self.mouse}, run {self.run_number}')
        run_path = os.path.join(self.pkl_path, self.mouse, f'run{self.run_number}.pkl')
        with open(run_path, 'rb') as f:  # load data
            r = pickle.load(f)
            self.run = r
        ## Start preprocessing:
        self.flu = self.run.flu
        self.tstart_galvo = utils.threshold_detect(self.run.x_galvo_uncaging, 0)
        self.tstart_galvo = self.run.spiral_start
        self.trial_start = self.run.trial_start
        assert len(self.trial_start) == len(self.tstart_galvo)
        self.galvo_ms = self.run.aligner.B_to_A(self.tstart_galvo)
        if verbose:
            print('microcontroller trial starts occur on average {} ms from galvo trial starts'
              .format(round(np.mean(self.trial_start - self.galvo_ms), 2)))

        ## Info about PS & iutcome
        ### a different number of cells were stimulated on each trial
        ### need to create a Subsets object to get this info (future code refinement will
        ### include this info directly in the run object
        self.subsets = Subsets(self.run)
        self.trial_subsets = self.subsets.trial_subsets
        self.n_stim_arr = np.unique(self.trial_subsets)
        self.outcome = self.run.outcome
        self.outcome_arr = np.unique(self.outcome)

    def define_s1_s2(self, im_size=1024):  # define border (which is hard-defined at middle of image)
        if self.run is None:
            self.load_data()
        self.n_cells = self.run.stat.shape[0]
        av_ypix = np.zeros(self.n_cells)
        av_xpix = np.zeros(self.n_cells)
        self.plane_number = np.zeros(self.n_cells)
        for neuron_id in range(self.n_cells):
            av_xpix[neuron_id] = np.mean(self.run.stat[neuron_id]['xpix']) % im_size  # modulo 1024 because different planes are transposed by image size
            av_ypix[neuron_id] = np.mean(self.run.stat[neuron_id]['ypix']) % im_size
            # JR addition to deal with single plane data that has no iplane
            try:
                self.plane_number[neuron_id] = self.run.stat[neuron_id]['iplane']
            except KeyError:
                self.plane_number[neuron_id] = 0
        self.s2_bool = av_ypix > 512
        self.s1_bool = np.logical_not(self.s2_bool)

    def build_trials_multi(self, pre_frames=20, post_frames=40, verbose=True):

        print(f'number of pre frames: {pre_frames}, number of post frames {post_frames}')

        assert self.run.frames_ms.shape == self.run.flu.shape
        self.pre_frames = pre_frames
        self.post_frames = post_frames
        self.art_gap_start = pre_frames - 1
        self.final_pre_gap_tp = np.arange(self.art_gap_start)[-1]
        self.art_gap_stop = pre_frames + 3
        self.filter_ps_array = np.concatenate((np.arange(self.art_gap_start), 
                                      np.arange(self.art_gap_stop, pre_frames + post_frames)))  # filter out few frames around PS

        # array of fluoresence through behavioural trials (n_cells x n_trials x n_frames)
        # with e.g. the first trials spanning (galvo_ms[0] - pre_frames) : (galvo_ms[0] + post_frames)
        self.behaviour_trials = utils.build_flu_array(self.run, self.galvo_ms, pre_frames, post_frames)
        self.behaviour_trials = self.behaviour_trials - np.nanmean(self.behaviour_trials, (1, 2))[:, np.newaxis, np.newaxis]
        print(f'Shape new array : {self.behaviour_trials.shape}')
        assert self.behaviour_trials.shape[1] == self.outcome.shape[0]


        self.pre_rew_trials = utils.build_flu_array(self.run, self.run.pre_reward, pre_frames, 
                                           pre_frames, is_prereward=True)  # equal amount b/c no PS artefact
        self.pre_rew_trials = self.pre_rew_trials[:, 1:9, :]
        assert np.sum(np.isnan(self.pre_rew_trials)) == 0
        self.pre_rew_trials = self.pre_rew_trials - np.mean(self.pre_rew_trials, (1, 2))[:, np.newaxis, np.newaxis]
        print(self.behaviour_trials.shape, self.pre_rew_trials.shape)
    
    def build_trials_single(self, pre_frames=20, post_frames=40, verbose=True):

        self.pre_frames = pre_frames
        self.post_frames = post_frames
        self.art_gap_start = pre_frames - 1
        self.final_pre_gap_tp = np.arange(self.art_gap_start)[-1]
        self.art_gap_stop = pre_frames + 3
        self.filter_ps_array = np.concatenate((np.arange(self.art_gap_start), 
                                      np.arange(self.art_gap_stop, pre_frames + post_frames)))  # filter out few frames around PS
        
        # array of fluoresence through behavioural trials (n_cells x n_trials x n_frames)
        # with e.g. the first trials spanning (galvo_ms[0] - pre_frames) : (galvo_ms[0] + post_frames)
        self.behaviour_trials = build_flu_array_single(self.run, pre_frames, post_frames, fs=30)
        self.behaviour_trials = self.behaviour_trials - np.nanmean(self.behaviour_trials, (1, 2))[:, np.newaxis, np.newaxis]
        print(f'Shape new array : {self.behaviour_trials.shape}')
        assert self.behaviour_trials.shape[1] == self.outcome.shape[0], '{} {}'.format(self.behaviour_trials.shape[1], self.outcome.shape[0])

        ### JR pre reward for single planes is not yet implmented ############
        
        #self.pre_rew_trials = utils.build_flu_array(self.run, self.run.pre_reward, pre_frames, 
        #                                   pre_frames, is_prereward=True)  # equal amount b/c no PS artefact
        #self.pre_rew_trials = self.pre_rew_trials[:, 1:9, :]
        #assert np.sum(np.isnan(self.pre_rew_trials)) == 0
        #self.pre_rew_trials = self.pre_rew_trials - np.mean(self.pre_rew_trials, (1, 2))[:, np.newaxis, np.newaxis]
        #print(self.behaviour_trials.shape, self.pre_rew_trials.shape)
           
        
    def label_trials(self):
        self.decision = np.logical_or(self.outcome == 'hit', self.outcome == 'fp').astype('int')
        self.photostim = np.ones_like(self.trial_subsets)  # ones = 5-50
        self.photostim[self.trial_subsets == 0] = 0
        self.photostim[self.trial_subsets == 150] = 2
        self.photostim_occ = {x: np.sum(self.photostim == x) for x in list(np.unique(self.photostim))}
        print(f'photo stim occurences: {self.photostim_occ}')
        self.autorewarded = np.array(rf.autoreward(self.run))  # array of bools whether an autoreward (after 3 consecutive misses) has occurred
        
        assert self.photostim.shape == self.decision.shape
        self.n_unique_stims = len(np.unique(self.photostim))
        self.n_neurons = self.behaviour_trials.shape[0]
        self.n_times = self.behaviour_trials.shape[2]
        self.n_trials = self.behaviour_trials.shape[1]
        self.n_unique_dec = len(np.unique(self.decision))
        self.occ_table = np.zeros((self.n_unique_stims, 2))  # stim x dec
        for dec in range(self.n_unique_dec):
            for stim in range(self.n_unique_stims):
                self.occ_table[stim, dec] = np.sum(np.logical_and(self.decision == dec, self.photostim == stim))
        self.n_com_trials = np.max(self.occ_table).astype('int')
        print('Occurence table:')
        print(self.occ_table)
    
    def remove_nan_trials_inplace(self, verbose=True):
        
        self.nonnan_trials = np.unique(np.where(~np.isnan(self.behaviour_trials))[1])
        self.behaviour_trials = self.behaviour_trials[:, self.nonnan_trials, :]
        self.photostim = self.photostim[self.nonnan_trials]
        self.decision = self.decision[self.nonnan_trials]
        self.trial_subsets = self.trial_subsets[self.nonnan_trials]
        self.outcome = self.outcome[self.nonnan_trials]
        self.autorewarded = self.autorewarded[self.nonnan_trials]
        
        if verbose:
            print(f'{len(self.nonnan_trials)} / {self.behaviour_trials.shape[1]} non nan trials identified')
            print(f'Numbers of PS cells: {np.unique(self.trial_subsets)}')  # exact amount of PS neurons
            print(f'Time array: {self.filter_ps_array}')  # time points outside of laser artefact

    def shuffle_labels(self):
        np.random.shuffle(self.photostim)
        np.random.shuffle(self.decision)
        np.random.shuffle(self.trial_subsets)
        np.random.shuffle(self.outcome)
        np.random.shuffle(self.autorewarded)

In [5]:
## Load data
sessions = {}

def only_numerics(seq):
    seq_type= type(seq)
    return seq_type().join(filter(seq_type.isdigit, seq))

def load_files(save_dict, data_dict, folder_path):
    total_ds = 0
    for mouse in data_dict.keys():
        for run_number in data_dict[mouse]:
            try:  # try/except framework to filter datasets that don't work - for whatever reason
                save_dict[total_ds] = Session(mouse, run_number, folder_path)
                total_ds += 1
                print(f'succesfully loaded mouse {mouse}, run {run_number}')
            except AttributeError:
                print(f'error in mouse {mouse}, run {run_number}')
    return sessions, total_ds


## Select all possible mouse/run combos:
# pkl_path = '/mnt/qnap_jrowland/run_pkls'
# all_mice = [x for x in os.listdir(pkl_path) if x[-4:] != '.pkl']
# run_dict = {m: list(np.unique([int(only_numerics(x)) for x in os.listdir(pkl_path + f'/{m}')])) for m in all_mice}
run_dict = {'J064' : [10],
            'RL048': [23, 24, 28]}
# sessions, total_ds = load_files(save_dict=sessions, data_dict=run_dict, folder_path=pkl_path)
# for _, ss in sessions.items():
#     ss.frequency = 5  # HARD set

pkl_path = '/home/jrowland/Documents/code/Vape/run_pkls'
all_mice = [x for x in os.listdir(pkl_path) if x[-4:] != '.pkl']
run_dict = {m: list(np.unique([int(only_numerics(x)) for x in os.listdir(pkl_path + f'/{m}')])) for m in all_mice}

sessions, total_ds = load_files(save_dict=sessions, data_dict=run_dict, folder_path=pkl_path)
for _, ss in sessions.items():
    if ss.frequency is None:
        ss.frequency = 30
        
sessions

Now loading mouse RL070, run 22
error in mouse RL070, run 22
Now loading mouse RL070, run 23
error in mouse RL070, run 23
Now loading mouse RL070, run 24
error in mouse RL070, run 24
Now loading mouse RL070, run 27
error in mouse RL070, run 27
Now loading mouse RL070, run 28
microcontroller trial starts occur on average -70.14 ms from galvo trial starts
Shape new array : (625, 386, 60)
photo stim occurences: {0: 127, 1: 140, 2: 119}
Occurence table:
[[123.   4.]
 [119.  21.]
 [ 80.  39.]]
379 / 379 non nan trials identified
Numbers of PS cells: [  0   5  10  20  30  40  50 150]
Time array: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 23 24 25 26 27
 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51
 52 53 54 55 56 57 58 59]
succesfully loaded mouse RL070, run 28
Now loading mouse RL070, run 29
microcontroller trial starts occur on average nan ms from galvo trial starts
Shape new array : (718, 279, 60)
photo stim occurences: {0: 96, 1: 93, 2: 90}
Occur

error in mouse J063, run 10
Now loading mouse J063, run 11
error in mouse J063, run 11
Now loading mouse J063, run 12
error in mouse J063, run 12
Now loading mouse J063, run 13
error in mouse J063, run 13
Now loading mouse J063, run 14
error in mouse J063, run 14
Now loading mouse RL023, run 5
error in mouse RL023, run 5
Now loading mouse RL023, run 7
error in mouse RL023, run 7
Now loading mouse RL023, run 8
error in mouse RL023, run 8
Now loading mouse RL023, run 9
error in mouse RL023, run 9
Now loading mouse RL023, run 11
error in mouse RL023, run 11
Now loading mouse RL023, run 12
error in mouse RL023, run 12
Now loading mouse RL048, run 14
error in mouse RL048, run 14
Now loading mouse RL048, run 23
microcontroller trial starts occur on average nan ms from galvo trial starts
number of pre frames: 20, number of post frames 40
Shape new array : (3571, 170, 60)
(3571, 170, 60) (3571, 8, 40)
photo stim occurences: {0: 62, 1: 52, 2: 56}
Occurence table:
[[56.  6.]
 [36. 16.]
 [20. 36.

{0: instance Mouse RL070, run 28 of Session class,
 1: instance Mouse RL070, run 29 of Session class,
 2: instance Mouse J048, run 27 of Session class,
 3: instance Mouse J048, run 28 of Session class,
 4: instance Mouse J048, run 29 of Session class,
 5: instance Mouse J048, run 30 of Session class,
 6: instance Mouse J048, run 32 of Session class,
 7: instance Mouse J064, run 10 of Session class,
 8: instance Mouse J064, run 11 of Session class,
 9: instance Mouse J064, run 14 of Session class,
 10: instance Mouse J063, run 8 of Session class,
 11: instance Mouse J063, run 9 of Session class,
 12: instance Mouse RL048, run 23 of Session class,
 13: instance Mouse RL048, run 24 of Session class,
 14: instance Mouse RL048, run 25 of Session class,
 15: instance Mouse RL048, run 28 of Session class,
 16: instance Mouse RL048, run 29 of Session class,
 17: instance Mouse J065, run 10 of Session class,
 18: instance Mouse J065, run 11 of Session class,
 19: instance Mouse J065, run 14 of 

In [6]:
run_dict

{'RL070': [22, 23, 24, 27, 28, 29],
 'J047': [7],
 'J048': [15, 16, 18, 19, 26, 27, 28, 29, 30, 31, 32],
 'RL022': [5, 7, 11, 13, 14],
 'J064': [10, 11, 12, 13, 14, 15],
 'RL072': [19, 20, 21, 24],
 'J063': [8, 9, 10, 11, 12, 13, 14],
 'RL023': [5, 7, 8, 9, 11, 12],
 'RL048': [14, 23, 24, 25, 28, 29],
 'RL032': [7, 10, 12, 13],
 'J065': [10, 11, 12, 13, 14, 15]}

In [8]:
sessions

{0: instance Mouse RL070, run 28 of Session class,
 1: instance Mouse RL070, run 29 of Session class,
 2: instance Mouse J048, run 27 of Session class,
 3: instance Mouse J048, run 28 of Session class,
 4: instance Mouse J048, run 29 of Session class,
 5: instance Mouse J048, run 30 of Session class,
 6: instance Mouse J048, run 32 of Session class,
 7: instance Mouse J064, run 10 of Session class,
 8: instance Mouse J064, run 11 of Session class,
 9: instance Mouse J064, run 14 of Session class,
 10: instance Mouse J063, run 8 of Session class,
 11: instance Mouse J063, run 9 of Session class,
 12: instance Mouse RL048, run 23 of Session class,
 13: instance Mouse RL048, run 24 of Session class,
 14: instance Mouse RL048, run 25 of Session class,
 15: instance Mouse RL048, run 28 of Session class,
 16: instance Mouse RL048, run 29 of Session class,
 17: instance Mouse J065, run 10 of Session class,
 18: instance Mouse J065, run 11 of Session class,
 19: instance Mouse J065, run 14 of 